# 🧬 Research / Production-Grade LLM_Variant_Pipeline

**Copyright (c) 2026 Ka-Kyung Kim**  
License: MIT  

GitHub: [kakyungkim](https://github.com/kakyungkim)



### 목적

* 공개 TCGA somatic VCF 샘플 기반 변이 분석
* 변이 데이터 정형화 및 LLM 기반 임상 해석 수행

### 전체 흐름

* GitHub에서 TCGA 샘플 파일 조회
* somatic VCF 다운로드 및 정리
* VCF → DataFrame 변환
* 샘플별 암종 정보 연결
* 주요 변이 선택
* LLM 기반 구조화된 해석 생성

---


In [ ]:
# 🧬 LLM_Variant_Pipeline
# Copyright (c) 2026 Ka-Kyung Kim
# License: MIT
# GitHub: https://github.com/kakyungkim
#
# This notebook demonstrates LLM-assisted somatic variant interpretation
# for research/demo purposes only. Not intended for clinical use.

## Environment & Setup

### Execution environment
- Google Colab
- Public TCGA data only

### Package installation

In [1]:
# 1️. 패키지 설치
# 기존 설치 제거 후, 지정 버전 설치
# !pip uninstall -y langchain langchain-core langchain-openai
!pip install -q \
  langchain-openai \
  pydantic \
  pandas

In [2]:
# 2. Library imports & environment check

# --- Standard library ---
import os
import sys
import glob
import gzip
import re
import tarfile
from io import BytesIO
from pathlib import Path
from getpass import getpass

# --- Third-party ---
import requests
import pandas as pd

import pydantic
from pydantic import BaseModel, Field
from typing import List, Optional

import langchain
import langchain_core
import langchain_openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Environment version logging (reproducibility)

import sys
from importlib.metadata import version, PackageNotFoundError

def get_pkg_version(pkg_name):
    try:
        return version(pkg_name)
    except PackageNotFoundError:
        return "not installed"

print("📦 Environment versions")
print("-----------------------")
print(f"Python           : {sys.version.split()[0]}")
print(f"langchain        : {get_pkg_version('langchain')}")
print(f"langchain-core   : {get_pkg_version('langchain-core')}")
print(f"langchain-openai : {get_pkg_version('langchain-openai')}")
print(f"pydantic         : {get_pkg_version('pydantic')}")
print(f"pandas           : {get_pkg_version('pandas')}")

📦 Environment versions
-----------------------
Python           : 3.12.12
langchain        : 0.2.0
langchain-core   : 0.2.11
langchain-openai : 0.1.14
pydantic         : 2.9.2
pandas           : 2.2.2


In [3]:
# 3. API Key 설정 (Colab 숨김)
# ⚠️ 실제 노트북에서는 직접 입력 또는 Secret Manager 사용 권장
API_KEY = getpass("Enter your OpenAI API Key: ")  # <--- 사용자 입력
os.environ["OPENAI_API_KEY"] = API_KEY
print("✅ API Key set!")

# 4. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 5. 프로젝트 폴더 경로 설정
PROJECT_DIR = Path("/content/drive/MyDrive/ToyProjects/LLM_Variant_Pipeline")
BASE_DIR = PROJECT_DIR / "data"
OUTPUT_DIR = PROJECT_DIR / "reports"

BASE_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"✅ Project directories ready!\nBASE_DIR: {BASE_DIR}\nOUTPUT_DIR: {OUTPUT_DIR}")


Enter your OpenAI API Key: ··········
✅ API Key set!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Project directories ready!
BASE_DIR: /content/drive/MyDrive/ToyProjects/LLM_Variant_Pipeline/data
OUTPUT_DIR: /content/drive/MyDrive/ToyProjects/LLM_Variant_Pipeline/reports


---

# Somatic Variant Interpretation Pipeline

## Single-Sample TCGA Variant Interpretation (LLM-Assisted)

> This notebook demonstrates a pipeline for interpreting somatic variants from a single tumor sample.
> Only a **single public TCGA sample** is used for testing and demonstration.
> Tier assignments are **candidate only** and require additional validation.



---

## 1. Introduction

### 목적

* Public TCGA somatic VCF 샘플을 입력으로 사용
* 단일 종양 샘플 내 변이를 구조화
* LLM을 이용해 **조건부 임상 해석(candidate tier)** 생성
* 이후 cohort 기반 분석으로 확장 가능한 파이프라인 설계

### 단일 샘플 분석의 한계

* cohort 빈도 정보 없음
* 통계적 driver 판별 불가
* 임상 tier 확정 불가

→ 본 노트북의 모든 tier는 **확정이 아닌 candidate 수준**

---

## 2. Data acquisition (public TCGA sample)

* TCGA Pan-Cancer 프로젝트의 공개 sample_files 사용
* GitHub API를 통해 파일 다운로드 (somatic SNV/MNV tar.gz 파일만 선별)
* `sample_files` 디렉토리 조회


In [4]:
import requests
from pathlib import Path

OWNER = "ICGC-TCGA-PanCancer"
REPO = "vcf-uploader"
PATH = "sample_files"
BRANCH = "develop"

# GitHub API로 sample_files 목록 조회
api_url = f"https://api.github.com/repos/{OWNER}/{REPO}/contents/{PATH}?ref={BRANCH}"
resp = requests.get(api_url)
resp.raise_for_status()
files = resp.json()

# 파일명에 'somatic'이 포함된 항목 필터링
somatic_urls = [
    f["download_url"]
    for f in files
    if "somatic" in f["name"].lower()
]

print(f"Number of somatic-related files to download: {len(somatic_urls)}")
for u in somatic_urls:
    print(" -", u.split("/")[-1])

# 다운로드
for url in somatic_urls:
    fname = url.split("/")[-1]
    out_path = Path(BASE_DIR) / fname
    print(f"Downloading {fname}...")
    r = requests.get(url)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        f.write(r.content)

print("Download complete at:", BASE_DIR)


Number of somatic-related files to download: 10
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.indel.vcf.gz
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.indel.vcf.gz.idx
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.indel.vcf.gz.idx.md5
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.indel.vcf.gz.md5
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.tar.gz
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.tar.gz.md5
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.vcf.gz
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.vcf.gz.idx
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.vcf.gz.idx.md5
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.vcf.gz.md5
Download compl

In [5]:
sorted(p.name for p in BASE_DIR.iterdir())

['a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.indel.vcf.gz',
 'a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.indel.vcf.gz.idx',
 'a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.indel.vcf.gz.idx.md5',
 'a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.indel.vcf.gz.md5',
 'a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.tar.gz',
 'a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.tar.gz.md5',
 'a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.vcf.gz',
 'a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.vcf.gz.idx',
 'a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.vcf.gz.idx.md5',
 'a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.vcf.gz.md5']

## 3. VCF parsing (SNV/MNV + INDEL)

### 분석 대상 기준

* 사용:

  * `.vcf.gz`

* 제외:

  * `.idx`, `.md5`, `.tar.gz`

* SNV/MNV와 INDEL을 파일명 기준으로 분리

In [6]:
vcf_files = list(BASE_DIR.glob("*.vcf.gz"))

snv_mnv_files = [f for f in vcf_files if "snv" in f.name.lower()]
indel_files   = [f for f in vcf_files if "indel" in f.name.lower()]

print("SNV/MNV files:")
for f in snv_mnv_files:
    print(" -", f.name)

print("\nINDEL files:")
for f in indel_files:
    print(" -", f.name)


SNV/MNV files:
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.snv_mnv.vcf.gz

INDEL files:
 - a4beedc3-0e96-4e1c-90b4-3674dfc01786.TestWorkflow_1-0-0.20141009.somatic.indel.vcf.gz


## 4. Variant structuring

### 설계 원칙

* 완전한 VCF 파싱은 목적 아님
* LLM 해석에 필요한 최소 정보만 유지
* INFO 필드는 문자열로 보존 (annotation 단계에서 활용)


In [7]:
import gzip
import pandas as pd

def parse_vcf_to_df(vcf_path, variant_type):
    rows = []

    with gzip.open(vcf_path, "rt") as f:
        for line in f:
            if line.startswith("#"):
                continue

            chrom, pos, _, ref, alt, _, _, info = line.strip().split("\t")[:8]

            rows.append({
                "chrom": chrom,
                "pos": int(pos),
                "ref": ref,
                "alt": alt,
                "variant_type": variant_type,
                "info": info
            })

    return pd.DataFrame(rows)


In [8]:
dfs = []

for f in snv_mnv_files:
    dfs.append(parse_vcf_to_df(f, "snv_mnv"))

for f in indel_files:
    dfs.append(parse_vcf_to_df(f, "indel"))

df_variants = pd.concat(dfs, ignore_index=True)
df_variants.head()


,chrom,pos,ref,alt,variant_type,info
0,1,14610,T,C,snv_mnv,FS=22.026;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPA...
1,1,14653,C,T,snv_mnv,FS=173.386;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMP...
2,1,14907,A,G,snv_mnv,FS=55.93;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPAC...
3,1,14930,A,G,snv_mnv,FS=31.945;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPA...
4,1,15118,A,G,snv_mnv,FS=3.583;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPAC...


## 5. Variant annotation (lightweight)

### 목적

* 외부 DB를 직접 붙이지 않고
* INFO 필드에서 최소한의 의미 있는 정보 추출
* gene 중심 해석을 가능하게 함

### 예시: INFO에서 GENE / SYMBOL 추출
* INFO 필드에 포함된 annotation tool 의존적 gene name 추출
* 현재 샘플은 SnpEff annotation 기준
* annotation tool이 변경되면 해당 파서만 교체

In [9]:
def extract_gene(info):
    match = re.search(r"SNPEFF_GENE_NAME=([^;]+)", info)
    return match.group(1) if match else None

df_variants["gene"] = df_variants["info"].apply(extract_gene)
df_variants.head()


,chrom,pos,ref,alt,variant_type,info,gene
0,1,14610,T,C,snv_mnv,FS=22.026;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPA...,DDX11L1
1,1,14653,C,T,snv_mnv,FS=173.386;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMP...,DDX11L1
2,1,14907,A,G,snv_mnv,FS=55.93;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPAC...,DDX11L1
3,1,14930,A,G,snv_mnv,FS=31.945;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPA...,DDX11L1
4,1,15118,A,G,snv_mnv,FS=3.583;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPAC...,DDX11L1


> 실제 확장 시:
>
> * VEP / ANNOVAR
> * COSMIC / ClinVar
> * OncoKB
>   등으로 교체 가능하도록 구조 유지

---

## 6. Candidate selection

### 목적

* 모든 변이를 LLM에 전달하지 않음
* 해석 예제에 적합한 소수 변이만 선택

### 단순 기준 (데모용)

* gene 정보가 있거나
* 변이 표현이 명확한 경우
* 상위 N개만 사용


In [10]:
MAX_VARIANTS = 10

df_candidates = (
    df_variants
    .sort_values(by=["gene"], na_position="last")
    .head(MAX_VARIANTS)
    .copy()
)

df_candidates

,chrom,pos,ref,alt,variant_type,info,gene
0,1,14610,T,C,snv_mnv,FS=22.026;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPA...,DDX11L1
17,1,16378,T,C,indel,FS=58.995;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPA...,DDX11L1
16,1,15274,A,T,indel,FS=0.0;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPACT=...,DDX11L1
15,1,15211,T,G,indel,FS=0.0;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPACT=...,DDX11L1
14,1,15118,A,G,indel,FS=3.583;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPAC...,DDX11L1
13,1,14930,A,G,indel,FS=31.945;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPA...,DDX11L1
12,1,14907,A,G,indel,FS=55.93;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPAC...,DDX11L1
11,1,14653,C,T,indel,FS=173.386;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMP...,DDX11L1
10,1,14610,T,C,indel,FS=22.026;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPA...,DDX11L1
9,1,16571,G,A,snv_mnv,FS=4.075;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPAC...,DDX11L1


### LLM 입력용 텍스트 변환

In [11]:
def variants_to_text(df):
    lines = []
    for _, r in df.iterrows():
        lines.append(
            f"{r.chrom}:{r.pos} {r.ref}>{r.alt} "
            f"[{r.variant_type}] "
            f"GENE={r.gene} INFO={r.info}"
        )
    return "\n".join(lines)

variant_text = variants_to_text(df_candidates)
print(variant_text)


1:14610 T>C [snv_mnv] GENE=DDX11L1 INFO=<bound method Series.info of chrom                                                           1
pos                                                         14610
ref                                                             T
alt                                                             C
variant_type                                              snv_mnv
info            FS=22.026;SNPEFF_GENE_NAME=DDX11L1;SNPEFF_IMPA...
gene                                                      DDX11L1
Name: 0, dtype: object>
1:16378 T>C [indel] GENE=DDX11L1 INFO=<bound method Series.info of chrom                                                           1
pos                                                         16378
ref                                                             T
alt                                                             C
variant_type                                                indel
info            FS=58.995;SNPEFF_GENE_NAME=DDX11

## 7. LLM-based interpretation (dual-mode tiering)
**Mode A: LLM-driven candidate tiering**
*   LLM이 변이별 candidate tier를 직접 판단
*   근거(evidence)와 함께 조건부로 기술
*   탐색적 분석, 신호 발굴에 적합

**Mode B: Rule-based pre-tiering + LLM explanation**
*   Tier는 rule-based 로직으로 사전에 결정
*   LLM은 tier를 변경하지 않고 설명만 생성
*   재현성, 리뷰 안전성, 공유 목적에 적합

### Tier 기준 구조화

Tier는 **확정이 아니라 조건부 판단**으로만 사용:

* `candidate_Tier_I`

  * well-known cancer gene
  * hotspot 또는 강한 oncogenic 근거
* `candidate_Tier_II`

  * oncogenic 가능성
  * 암 관련 경로 연관
* `uncertain`

  * 근거 부족 또는 정보 제한

---

### (1) 출력 스키마 정의 (Pydantic)


In [12]:
from pydantic import BaseModel
from typing import List, Optional

class VariantInterpretation(BaseModel):
    gene: Optional[str]
    variant: str
    variant_type: str

    oncogenic_evidence: str
    potential_tier: str
    tier_basis: str

    requires_additional_validation: bool
    notes: Optional[str]


class SampleInterpretation(BaseModel):
    cancer_context: Optional[str]

    variants: List[VariantInterpretation]

    overall_summary: str
    limitations: str



---
### (2) LLM + Parser 설정

In [13]:
# from langchain_openai import ChatOpenAI
# from langchain_core.output_parsers import JsonOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = JsonOutputParser(pydantic_object=SampleInterpretation)

---

### (3) 프롬프트 템플릿 정의

In [14]:
# Mode 선택용 설정값
TIER_MODE = "llm"      # 또는 "rule"

In [15]:
from langchain_core.prompts import ChatPromptTemplate

def build_prompt_template(tier_mode: str) -> ChatPromptTemplate:
    if tier_mode == "llm":
        tier_instruction = """
You may assign a candidate tier based on available evidence.
Use conditional language only.
"""
    elif tier_mode == "rule":
        tier_instruction = """
Candidate tier has been pre-assigned using rule-based logic.
Do NOT change the tier. Only provide explanation.
"""
    else:
        raise ValueError("tier_mode must be 'llm' or 'rule'")

    return ChatPromptTemplate.from_template(f"""
You are interpreting somatic variants from a single tumor sample.

Constraints:
- Single-sample analysis only
- No cohort-level frequency data
- No definitive clinical decisions

{tier_instruction}

Variants:
{{variant_text}}

Cancer context:
{{cancer_context}}

Tier definitions:
- candidate_Tier_I: strong known oncogenic relevance
- candidate_Tier_II: moderate or emerging evidence
- uncertain: insufficient evidence

Output only valid JSON matching this schema:
{{format_instructions}}
""")

---

### (4) 템플릿 → 메시지 → 실행

In [16]:
prompt_tmpl = build_prompt_template(TIER_MODE)

messages = prompt_tmpl.format_messages(
    variant_text=variant_text,
    cancer_context="pancreatic adenocarcinoma",
    format_instructions=parser.get_format_instructions()
)

# 1) 파이프라인 메타데이터
# NOTE:
# TCGA sample identifier is inferred from VCF filename.
# In multi-sample or production settings, explicit sample metadata should be used.
SAMPLE_ID = snv_mnv_files[0].stem

# 2) LLM 실행
result = llm.invoke(messages)
parsed = parser.parse(result.content)

# 3) LLM 결과 + sample_id 결합
final_result = {
    "sample_id": SAMPLE_ID,
    "cancer_context": "pancreatic adenocarcinoma",
    "variants": parsed["variants"],
    "overall_summary": parsed["overall_summary"],
    "limitations": parsed["limitations"]
}

# 7.1 — Dual-mode comparison (Optional analysis)

> 목적
>
> * **같은 변이 집합**에 대해
>
>   * Mode A: LLM이 tier까지 판단
>   * Mode B: Rule-based tier 고정 + LLM 설명
> * 결과를 **나란히 비교**
> * “LLM이 왜 tier를 판단해도 되는가”를 데이터로 보여줌


```
VCF (SNV / INDEL)
        │
        ▼
Lightweight Parsing
        │
        ▼
Candidate Variant Selection
        │
        ├──────────────┐
        ▼              ▼
 Mode A (LLM)      Mode B (Rule)
 Tier + Evidence   Tier fixed
        │              │
        └───────┬──────┘
                ▼
        Dual-mode Comparison
                │
                ▼
        Structured JSON / CSV
```


---

## 7.1-A. 공통 준비 (이미 7번에서 만든 것 재사용)

In [17]:
# 공통 메타데이터
SAMPLE_ID = snv_mnv_files[0].stem
CANCER_CONTEXT = "pancreatic adenocarcinoma"

---

## 7.1-B. Mode A — LLM-driven candidate tiering

### (1) Prompt 생성

In [18]:
prompt_llm = build_prompt_template("llm")

messages_llm = prompt_llm.format_messages(
    variant_text=variant_text,
    cancer_context=CANCER_CONTEXT,
    format_instructions=parser.get_format_instructions()
)

### (2) 실행

In [19]:
result_llm = llm.invoke(messages_llm)
parsed_llm = parser.parse(result_llm.content)

---

## 7.1-C. Mode B — Rule-based pre-tiering + LLM explanation

### (1) Rule-based tier 정의 (데모용)

In [20]:
# def assign_candidate_tier(row):
#     """
#     Demo-only rule-based pre-tiering.
#     Not intended for clinical or guideline-compliant use.
#     """
#     if row["gene"] in {"KRAS", "EGFR", "BRAF"}:
#         return "candidate_Tier_I"
#     elif row["gene"] is not None:
#         return "candidate_Tier_II"
#     else:
#         return "uncertain"

# df_candidates["rule_based_tier"] = df_candidates.apply(
#     assign_candidate_tier, axis=1
# )

In [21]:
# Minimal cancer-relevant gene set (demo)
KNOWN_ONCOGENES = {
    "KRAS", "NRAS", "HRAS",
    "EGFR", "BRAF", "PIK3CA",
    "TP53", "CDKN2A", "SMAD4"
}

def assign_candidate_tier(row):
    """
    Rule-based candidate tiering (INTENTIONALLY MINIMAL).

    Conceptual mapping to AMP/ASCO/CAP guidelines:
    - candidate_Tier_I   ~ AMP Tier I/II (strong clinical or biological evidence)
    - candidate_Tier_II  ~ AMP Tier III (biological relevance, limited evidence)
    - uncertain          ~ AMP Tier IV (unknown significance)

    This rule is NOT guideline-compliant and is used only
    to provide a stable comparison baseline for LLM behavior.
    """

    gene = row["gene"]

    # Well-established oncogenes / tumor suppressors
    if gene in KNOWN_ONCOGENES:
        return "candidate_Tier_I"

    # Missing or undefined gene
    if gene is None:
        return "uncertain"

    # Pseudogenes / uncharacterized loci (very coarse heuristic)
    if gene.endswith("L") or gene.startswith("LOC"):
        return "uncertain"

    # Fallback: gene-level relevance but no strong evidence
    return "candidate_Tier_II"


df_candidates = df_candidates.copy()
df_candidates["rule_based_tier"] = df_candidates.apply(
    assign_candidate_tier, axis=1
)

# (선택) sanity check
df_candidates[["gene", "rule_based_tier"]].head()

,gene,rule_based_tier
0,DDX11L1,candidate_Tier_II
17,DDX11L1,candidate_Tier_II
16,DDX11L1,candidate_Tier_II
15,DDX11L1,candidate_Tier_II
14,DDX11L1,candidate_Tier_II


### (2) Rule tier 포함한 variant text 생성

In [22]:
def variants_to_text_with_rule(df):
    lines = []
    for _, r in df.iterrows():
        lines.append(
            f"{r['chrom']}:{r['pos']} {r['ref']}>{r['alt']} "
            f"[{r['variant_type']}] "
            f"GENE={r['gene']} PRE_TIER={r['rule_based_tier']}"
        )
    return "\n".join(lines)

variant_text_rule = variants_to_text_with_rule(df_candidates)
print(variant_text_rule)


1:14610 T>C [snv_mnv] GENE=DDX11L1 PRE_TIER=candidate_Tier_II
1:16378 T>C [indel] GENE=DDX11L1 PRE_TIER=candidate_Tier_II
1:15274 A>T [indel] GENE=DDX11L1 PRE_TIER=candidate_Tier_II
1:15211 T>G [indel] GENE=DDX11L1 PRE_TIER=candidate_Tier_II
1:15118 A>G [indel] GENE=DDX11L1 PRE_TIER=candidate_Tier_II
1:14930 A>G [indel] GENE=DDX11L1 PRE_TIER=candidate_Tier_II
1:14907 A>G [indel] GENE=DDX11L1 PRE_TIER=candidate_Tier_II
1:14653 C>T [indel] GENE=DDX11L1 PRE_TIER=candidate_Tier_II
1:14610 T>C [indel] GENE=DDX11L1 PRE_TIER=candidate_Tier_II
1:16571 G>A [snv_mnv] GENE=DDX11L1 PRE_TIER=candidate_Tier_II


### (3) Prompt 생성

In [23]:
prompt_rule = build_prompt_template("rule")

messages_rule = prompt_rule.format_messages(
    variant_text=variant_text_rule,
    cancer_context=CANCER_CONTEXT,
    format_instructions=parser.get_format_instructions()
)

### (4) 실행

In [24]:
result_rule = llm.invoke(messages_rule)
parsed_rule = parser.parse(result_rule.content)

---

## 7.1-D. 결과 통합 (sample_id 포함)

In [25]:
dual_mode_result = {
    "sample_id": SAMPLE_ID,
    "cancer_context": CANCER_CONTEXT,
    "llm_mode": parsed_llm,
    "rule_mode": parsed_rule
}

---

## 7.1-E. Dual-mode 비교 테이블 생성

> 핵심: **variant 단위로 tier 판단이 어떻게 달라지는지**

In [26]:
# --- 1) variant key 정의 함수 ---
def make_variant_key(v):
    return f"{v['variant']}|{v['variant_type']}"

# --- 2) LLM / Rule 결과를 dict로 변환 ---
llm_map = {
    make_variant_key(v): v
    for v in parsed_llm["variants"]
}

rule_map = {
    make_variant_key(v): v
    for v in parsed_rule["variants"]
}

# --- 3) 공통 key 기준으로 비교 ---
rows = []

for key in llm_map.keys() & rule_map.keys():
    v_llm = llm_map[key]
    v_rule = rule_map[key]

    rows.append({
        "gene": v_llm.get("gene"),
        "variant": v_llm["variant"],
        "variant_type": v_llm["variant_type"],
        "LLM_candidate_tier": v_llm["potential_tier"],
        "Rule_candidate_tier": v_rule["potential_tier"],
        "LLM_tier_basis": v_llm["tier_basis"],
        "Rule_tier_basis": v_rule["tier_basis"],
        "LLM_requires_validation": v_llm["requires_additional_validation"]
    })

comparison_df = pd.DataFrame(rows)
comparison_df["tier_match"] = (
    comparison_df["LLM_candidate_tier"]
    == comparison_df["Rule_candidate_tier"]
)

In [27]:
concordant_df = comparison_df.loc[comparison_df["tier_match"]]
concordant_df

,gene,variant,variant_type,LLM_candidate_tier,Rule_candidate_tier,LLM_tier_basis,Rule_tier_basis,LLM_requires_validation,tier_match


In [28]:
discordant_df = comparison_df.loc[~comparison_df["tier_match"]]
discordant_df

,gene,variant,variant_type,LLM_candidate_tier,Rule_candidate_tier,LLM_tier_basis,Rule_tier_basis,LLM_requires_validation,tier_match
0,DDX11L1,1:14907 A>G,indel,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False
1,DDX11L1,1:14610 T>C,indel,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False
2,DDX11L1,1:15118 A>G,indel,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False
3,DDX11L1,1:14610 T>C,snv_mnv,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False
4,DDX11L1,1:15274 A>T,indel,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False
5,DDX11L1,1:15211 T>G,indel,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False
6,DDX11L1,1:16571 G>A,snv_mnv,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False
7,DDX11L1,1:14653 C>T,indel,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False
8,DDX11L1,1:16378 T>C,indel,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False
9,DDX11L1,1:14930 A>G,indel,uncertain,candidate_Tier_II,Insufficient evidence for oncogenic relevance.,Pre-assigned based on rule-based logic.,True,False


### CSV 저장

In [29]:
comparison_df.to_csv(
    OUTPUT_DIR / f"{SAMPLE_ID}_dual_mode_comparison.csv",
    index=False
)

## 8. Output & limitations

### 출력 특징

* 변이별:

  * oncogenic 근거 설명
  * tier 근거 명시
  * 추가 검증 필요 여부
* 샘플 수준 요약 포함
* 한계 사항을 명시적으로 기술

이 구조는:

* 과장된 임상 주장 방지
* 리뷰/공유에 안전

---

## 9. Future extensions

* 다중 샘플 → cohort 분석
* VAF 기반 필터링
* COSMIC / ClinVar / OncoKB 연동
* AMP/ASCO/CAP guideline 정식 매핑
* LLM 출력 → 리포트 자동 생성

---